In [1]:
import time as t
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timezone
from os import listdir
from os.path import isfile, join
import re
import pytz

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import gc
#% matplotlib inline

In [2]:
'''
##########################################################################################################
1. FUNCTIONS: General Support Function
##########################################################################################################
'''
def checkLogType(series):
    import random
    diff = series.diff().dropna()
    diff = diff.reset_index(drop=True)
    randList = random.sample(range(len(diff)), 3)
    if diff[randList].iloc[0] == diff[randList].iloc[1] == diff[randList].iloc[2]:
        return 'state'
    else:
        return 'event'

def dtFormatInspector(dt, fmt):
    try:
        pd.to_datetime(dt, format=fmt)
        return True
    except:
        return False

def dtFormatString(dtcol):
    if dtFormatInspector(dtcol[5], '%d/%m/%Y %H:%M') and dtFormatInspector(dtcol[dtcol.index[-5]], '%d/%m/%Y %H:%M'):
        fmtString = '%d/%m/%Y %H:%M'
    elif dtFormatInspector(dtcol[5], '%m/%d/%Y %H:%M') and dtFormatInspector(dtcol[dtcol.index[-5]], '%m/%d/%Y %H:%M'):
        fmtString = '%m/%d/%Y %H:%M'
    elif dtFormatInspector(dtcol[5], '%m/%d/%Y %H:%M:%S'):
        fmtString = '%m/%d/%Y %H:%M:%S'
    elif dtFormatInspector(dtcol[5], '%m/%d/%Y %I:%M %p'):
        fmtString = '%m/%d/%Y %I:%M %p'
    elif dtFormatInspector(dtcol[5], '%m/%d/%Y %I:%M:%S %p'):
        fmtString = '%m/%d/%Y %I:%M:%S %p'       
    elif dtFormatInspector(dtcol[5], '%m/%d/%y %H:%M'):
        fmtString = '%m/%d/%y %H:%M'
    elif dtFormatInspector(dtcol[5], '%m/%d/%y %H:%M:%S'):
        fmtString = '%m/%d/%y %H:%M:%S'    
    elif dtFormatInspector(dtcol[5], '%m/%d/%y %I:%M %p'):
        fmtString = '%m/%d/%y %I:%M %p'
    elif dtFormatInspector(dtcol[5], '%m/%d/%y %I:%M:%S %p'):
        fmtString = '%m/%d/%y %I:%M:%S %p'    
    elif dtFormatInspector(dtcol[5], '%m/%d/%y %H:%M:%S.%f'):
        fmtString = '%m/%d/%y %H:%M:%S.%f'      
    elif dtFormatInspector(dtcol[5], '%Y-%m-%d %H:%M:%S'):
        fmtString = '%Y-%m-%d %H:%M:%S'
    elif dtFormatInspector(dtcol[5], '%Y-%m-%d %I:%M:%S %p'):
        fmtString = '%Y-%m-%d %I:%M:%S %p' 
    elif dtFormatInspector(dtcol[5], '%Y/%m/%d %H:%M:%S'):
        fmtString = '%Y/%m/%d %H:%M:%S'
    elif dtFormatInspector(dtcol[5], '%d-%m-%Y,%H:%M:%S'):
        fmtString = '%d-%m-%Y,%H:%M:%S'   
    elif dtFormatInspector(dtcol[5], '%d-%m-%Y%H:%M:%S'):
        fmtString = '%d-%m-%Y%H:%M:%S'
    elif dtFormatInspector(dtcol[5], '%Y-%m-%d%H:%M:%S'):
        fmtString = '%Y-%m-%d%H:%M:%S'
    elif dtFormatInspector(dtcol[5], '%m-%d-%Y %H:%M'):
        fmtstring = '%m-%d-%Y %H:%M'
    else:
        print('datetime format error')
    return fmtString    

def extractName(fpath):
    if fpath[-4:].lower() == '.csv' or fpath[-4:].lower() == '.xls':
        fname = fpath[:-4]
    elif fpath[-5:].lower() == '.xlsx':
        fname = fpath[:-5]
    elif fpath[-4:].lower() == '.txt':
        fname = fpath[:-4]
    return fname

def findHeader(fpath, lookup):
    if fpath[-4:] == '.csv':
        with open(fpath) as f:
            for num, line in enumerate(f, 1):
                if lookup in line:
                    rowNum = num   
    elif fpath[-5:] == '.xlsx':
        df = pd.read_excel(fpath, header=None, skiprows=0, index_col=False)
        rowNum = df.loc[df.iloc[:,0]==lookup].index[0]  
    return rowNum

def fromExcelDate(dtcol):
    # All credit to StackOverflow for this function:
    # https://stackoverflow.com/questions/29387137/...
    # ...how-to-convert-a-given-ordinal-number-from-excel-to-a-date
    
    # dtcol MUST be a series    
    epoch0 = pd.to_datetime('1899-12-31')
    
    # Excel leap year bug, 1900 is not a leap year!  
    dtcol = dtcol.astype(float)
    dtcol.where(dtcol<=59, dtcol-1, inplace=True)          
    td = pd.to_timedelta(dtcol, unit='days')
    return (epoch0 + td).dt.round('1s')

def temperatureConverter(temp, current_units):
    if current_units.lower() == 'f':
        # convert from Fahrenheit to celsius
        celsius = (temp - 32) * (5/9)
        output = celsius
    elif current_units.lower() == 'c':
        # convert from celsius to fahrenheit
        fahrenheit = (temp * (9/5)) + 32
        output = fahrenheit
    return output

def fileInspector(prefix, rawF, suffix):  
    is_instrFile = lambda f: f.startswith(prefix) and f.lower().endswith(suffix)
    fList = [f for f in listdir(rawF) if isfile(join(rawF,f)) and is_instrFile(f)]
    return fList

def folderCrawler(rawF, home_id, week=None):
    if week == 1:
        test = 'ON'
#         fmtIBU = '{}-{}-Week{}-{}-'
    elif week == 2:
        test = 'OFF'
#         fmtIBU = '{}-{}-Week{}-{}-'
    elif week == None:
        test = None
#         fmtIBU = '{}-{}-'
    instrKeys = ['ANM','AVP','CLR','FRM','IBU','MTR','PLD','PMI','PMO',
                 'RAD','RDSTR','STA','TRH','TRO','UFP','WTR','WX']
    instrValues = []
    formatDict = {i:'{}-{}-' for i in instrKeys}
#     formatDict['IBU'] = fmtIBU    
#     if home_id.startswith('3'):
#         formatDict['PMO'] = '{}-{}-Smp1Min'
    for instr in instrKeys:
        template = formatDict[instr]       
        if instr in set(['AVP','ANM','FRM','MTR','PLD','PMI','PMO','STA','TRH','TRO','RAD','RDSTR','WTR','WX','CLR','UFP']):            
            prefix = template.format(home_id, instr)            
            suffix = ('.csv','.xls', '.xlsx','.txt')           
            fList = fileInspector(prefix, rawF, suffix)
            instrValues.append(fList)            
        elif instr in set(['IBU']):
            if week == 1 or week == 2:
                prefix = template.format(home_id, instr, week, test)
            elif week == None:
                prefix = template.format(home_id, instr)
            suffix = ('.csv','.xls','.xlsx')  
            fList = fileInspector(prefix, rawF, suffix)
            instrValues.append(fList) 
        del template, suffix        
    instrDict = dict(zip(instrKeys, instrValues))
    return (instrDict, test)



In [3]:
print('./')

./


In [4]:
'''
##########################################################################################################
2. Input: General Setup
##########################################################################################################
'''
# Set HomeID and Week#
home_id = '224'
week = 1

##############################################################################################
# Specify folders
rawF = './Home_' + home_id + '/' # Raw files location
tsF = './' # read time period from HomeCharacteristics file
outF = './' # DataTable output folder
if int(home_id) in range(400,500):
    ts_fname = 'HomeCharacteristics_Output_401_425.xlsx'
elif int(home_id) in range(200,400):
    ts_fname = 'HomeCharacteristics_Output.xlsx' 
elif int(home_id) in range(500,600):
    ts_fname = 'HomeCharacteristics_Output_ICRT.xlsx'
    
if int(home_id) in set(range(900,1000)): # KVZNE study
        startrow = 'Field_StartDate'
        stoprow = 'Field_EndDate'
elif int(home_id) in set(range(200,600)): # BAIAQ study
        if week == 1 or week == None:
            startrow = 'Field_StartDate_Week1'
            stoprow = 'Field_EndDate_Week1'
        elif week == 2:
            startrow = 'Field_StartDate_Week2'
            stoprow = 'Field_EndDate_Week2'
else:
    print("change home_id")

##############################################################################################
# Load/Filter DataFrames 
# daylight saving not considered, change stophour to enable manual adjustment in datatables
ts_df = pd.read_excel(tsF + ts_fname, header=0, index_col=0)

start = pd.to_datetime(ts_df.loc[startrow, int(home_id)], format = '%Y-%m-%d') + pd.Timedelta(hours=18)
stop = pd.to_datetime(ts_df.loc[stoprow, int(home_id)], format = '%Y-%m-%d') + pd.Timedelta(hours=9)
startday = start.strftime('%Y-%m-%d')
stopday = stop.strftime('%Y-%m-%d')

df_index = pd.DataFrame(index = pd.date_range(start=start, end=stop, freq='T'))
dataTable = df_index

##############################################################################################
# establish data file list
instrDict, test = folderCrawler(rawF, home_id, week)

##############################################################################################
# Print stuff
print('Home {} {}'.format(home_id, test))
print(str(start) + ' to ' + str(stop))
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
count = 0
for x in instrDict:
    print (str(x) + ':')
    for y in instrDict[x]:
        print('  '+ str(y))
        count += 1
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print(str(count) + ' files')

Home 224 ON
2019-12-30 18:00:00 to 2019-01-06 09:00:00
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ANM:
  224-ANM-BA2.xls
  224-ANM-BA1.xls
  224-ANM-KIT.xls
  224-ANM-AS1.xls
AVP:
CLR:
FRM:
  224-FRM-BR1.csv
  224-FRM-IN1.csv
IBU:
  224-IBU-FP1.csv
  224-IBU-COF.csv
  224-IBU-OV1.csv
  224-IBU-CLF.csv
  224-IBU-CLB.csv
  224-IBU-TOA.csv
  224-IBU-CRB.csv
  224-IBU-CRF.csv
MTR:
  224-MTR-CDR.csv
PLD:
PMI:
  224-PMI-IN1.CSV
PMO:
RAD:
  224-RAD-IN1.csv
RDSTR:
STA:
  224-STA-DGH.csv
  224-STA-DP1.csv
  224-STA-BR1.csv
  224-STA-DFR.csv
TRH:
  224-TRH-BR1.csv
  224-TRH-AS1.csv
TRO:
  224-TRO-BA2.csv
  224-TRO-BA1.csv
  224-TRO-OUT.csv
UFP:
WTR:
WX:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
26 files


In [11]:
'''
##########################################################################################################
3. FUNCTIONS: Reading and Processing raw data files
##########################################################################################################
'''
def readFile(f, instr):
    if instr == 'AVP':
        df = read_AVP(f)
    elif instr == 'ANM':
        df = read_ANM(f)
    elif instr == 'CLR':
        df = read_CLR(f)
    elif instr == 'FRM':
        df = read_FRM(f)
    elif instr == 'IBU':
        df = read_IBU(f)
    elif instr == 'MTR':
        df = read_MTR(f)
    elif instr == 'PLD':
        df = read_PLD(f)
    elif instr == 'PMI':
        df = read_PMI(f)
    elif instr == 'PMO':
        df = read_PMO(f)
    elif instr == 'STA':
        df = read_STA(f)
    elif instr == 'TRH' or instr == 'TRO':
        df = read_TRH(f)
    elif instr == 'UFP':
        df = read_UFP(f)
    elif instr == 'WTR' or instr == 'WX':
        df = read_WTR(f)
    elif instr == 'RAD' or instr == 'RDSTR':
        df = read_RAD(f)
    return df

def processFile(f, ind, instr, home_id, week=1):
    if instr == 'AVP':
        df = process_AVP(f, ind)
    elif instr == 'ANM':
        df = process_ANM(f, ind)
    elif instr == 'FRM':
        df = process_FRM(f, ind)
    elif instr == 'IBU':
        df = process_IBU(f, ind)
    elif instr == 'MTR':
        df = process_MTR(f, ind)
    elif instr == 'PLD':
        df = process_PLD(f, ind)
    elif instr == 'PMI':
        df = process_PMI(f, ind)
    elif instr == 'PMO':
        df = process_PMO(f, ind)
    elif instr == 'STA':
        df = process_STA(f, ind)
    elif instr == 'TRH' or instr == 'TRO':
        df = process_TRH(f, ind)
    elif instr == 'RAD' or instr == 'RDSTR':
        df = process_RAD(f, ind)
    elif instr == 'WTR' or instr == 'WX':
        df = process_WTR(f,ind)
    elif instr == 'CLR':
        df = process_CLR(f,ind)
    elif instr == 'UFP':
        df = process_UFP(f,ind)
    return df

##############################################################################################
# 3.1 ANM - ANEMOMETER: 30sec data (rolling average)
# insert last row with "NA" for 901
##############################################################################################

def read_ANM(ANMfile): # .csv, .xls, .xlsx, insert last row as "NA" if endhour earlier than 9am
    readParamsDict = {'skiprows': 6,'index_col': False}
    header_ANM = ['id', 'velocity_mps', 'unit', 'date', 'time']

    if ANMfile.endswith("csv"):    
        ANM = pd.read_csv(rawF + ANMfile, names=header_ANM, **readParamsDict)
        ANM = ANM.apply(lambda x: x.replace(' ', '', regex=True) if x.dtype == "object" else x)
    elif ANMfile.endswith("xls"):
        ANM = pd.read_csv(rawF + ANMfile, names=header_ANM, sep='\t|,', engine='python', **readParamsDict)
        ANM = ANM.apply(lambda x: x.replace(' ', '', regex=True) if x.dtype == "object" else x)
        ANM.iloc[:,[3,4]] = ANM.iloc[:,[3,4]].apply(lambda x: x.str.strip('"'))     
    elif ANMfile.endswith("xlsx"):
        df = pd.read_excel(rawF + ANMfile, header=None, skiprows=0, index_col=False)
        rows = findHeader(rawF + ANMfile, 'ID')
        try:
            header_ANM = ['id', 'velocity_mps', 'unit', 'datetime']
            ANM = pd.read_excel(rawF + ANMfile, names=header_ANM, header=0, skiprows=rows, index_col=False)
        except:
            header_ANM = ['id', 'velocity_mps', 'unit', 'date', 'time']
            ANM = pd.read_excel(rawF + ANMfile, names=header_ANM, header=0, skiprows=rows, index_col=False)       
        ANM = ANM.apply(lambda x: x.replace(' ', '', regex=True) if x.dtype == "object" else x)
    else:
        print("ANM file format error")
    
    if ANM.loc[:,'unit'][0].lower() == 'ft/min':
        ANM.loc[:,'velocity_mps'] = ANM.loc[:,'velocity_mps'] * 0.00507999983744
        ANM.loc[:,'unit'] = 'm/s'
    elif ANM.loc[:,'unit'][0].lower() != 'ft/min' and ANM.loc[:,'unit'][0].lower() != 'm/s':
        print("ANM unit error")
    
    if len(ANM.columns) == 4:
        ANM['datetime'] = pd.to_datetime(ANM['datetime'], format=dtFormatString(ANM['datetime']))
    else:
        fmt = dtFormatString(ANM['date'].astype(str) + ANM['time'].astype(str))
        ANM.loc[:,'datetime'] = pd.to_datetime(ANM['date'].astype(str) + ANM['time'].astype(str), format=fmt)
    
    if ANM['datetime'].iloc[-1] < stop:
        newdatetime = ANM['datetime'].iloc[-1]+pd.Timedelta(minutes=1)
        newrow = pd.Series({'datetime' : newdatetime , 'velocity_mps' : None})
        ANM = ANM.append(newrow,ignore_index = True)
    
    ANM.index = ANM['datetime']  
    ANM = ANM['velocity_mps']    
    return ANM


def process_ANM(ANMfile, ind):
    fname = extractName(ANMfile)    
    home_id, instr, loc = fname.split('-',2)
    pad = 600 # seconds
    rawANM = read_ANM(ANMfile)
    rawANM = rawANM[(rawANM.index>start-pd.Timedelta(seconds=pad)) & (rawANM.index<stop+pd.Timedelta(seconds=pad))]
    
    ANM = pd.merge_asof(ind, rawANM, left_index=True, right_index=True, direction='backward')    
    colmapper = {'velocity_mps':'{}_{}'.format(instr, loc)}
    ANM.rename(columns = colmapper, inplace=True)
    ANM.fillna('NA', inplace=True)
    return ANM


##############################################################################################
# 3.2 AVP - AIR VISUAL PRO: 10sec-5min data (rolling average)
##############################################################################################
def read_AVP(AVPfile): # .csv, .xlsx
    home_id, instr, loc = re.split('-', AVPfile[:-4])[0:3]
    if AVPfile.endswith('xlsx'):        
        AVP = pd.read_excel(rawF + AVPfile, header=0, index_col=False)
    elif AVPfile.endswith('csv'):
        AVP = pd.read_csv(rawF+AVPfile, sep=',')
    elif AVPfile.endswith('txt'):
        AVP = pd.read_csv(rawF+AVPfile, sep=';')
    else:
        print('AVP file format error')
    
    AVP['datetime'] = pd.to_datetime(AVP.Date.astype(str) + ' ' + AVP.Time.astype(str),errors='coerce')
    AVP = AVP[AVP['datetime']<stop+pd.Timedelta(days=1)]
    AVP = AVP[['datetime','PM2_5(ug/m3)','PM10(ug/m3)','Temperature(C)','Humidity(%RH)','CO2(ppm)']]
    AVP.columns = ['datetime','pm25_ugm3','pm10_ugm3','temp_degC','rh_pct','co2_ppm']
    AVP = AVP[AVP['pm25_ugm3']!= 1798.8]
    AVP = AVP[AVP['pm10_ugm3']!= 2000]
    if AVP['datetime'].iloc[-1] < stop:
        newdatetime = AVP['datetime'].iloc[-1]+pd.Timedelta(minutes=1)
        newrow = pd.Series({'datetime' : newdatetime})
        AVP = AVP.append(newrow,ignore_index = True)  
    AVP = AVP.drop_duplicates(subset='datetime',keep = 'first')    
    AVP.index = AVP['datetime']
    AVP = AVP[['pm25_ugm3','pm10_ugm3','temp_degC','rh_pct','co2_ppm']]
    return AVP


def process_AVP(AVPfile, ind):
    fname = extractName(AVPfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    pad = 600 # seconds
    rawAVP = read_AVP(AVPfile)
    rawAVP = rawAVP[(rawAVP.index>start-pd.Timedelta(seconds=pad)) & (rawAVP.index<stop+pd.Timedelta(seconds=pad))]

    rawAVP = rawAVP.dropna()
    rawAVP[['pm10_ugm3','rh_pct','co2_ppm']] = rawAVP[['pm10_ugm3','rh_pct','co2_ppm']].astype(np.float)
    smAVP = rawAVP.resample('s').interpolate(method='time', axis=0, limit_area='inside')
    smAVP.index = smAVP.index.shift(-1, freq='30s')
    smAVP = smAVP.rolling('60s').mean()
    AVP = pd.merge(ind, smAVP, left_index=True, right_index=True, how='left')
    
    colmapper = {'pm25_ugm3':'{}_{}_PM25'.format(instr, loc),
                 'pm10_ugm3':'{}_{}_PM10'.format(instr, loc),
                 'co2_ppm':'{}_{}_CO2'.format(instr, loc),
                 'temp_degC':'{}_{}_T'.format(instr, loc),
                 'rh_pct':'{}_{}_RH'.format(instr, loc)}
    
    AVP.rename(columns = colmapper, inplace=True)
    AVP.fillna('NA', inplace=True)
    return AVP


##############################################################################################
# 3.3 CLR - Clarity: UTC, 3min data, pm1/2.5/10/no2(all raw),temp,RH,tVoc,eCo2 (interpolation)
##############################################################################################
def read_CLR(CLRfile):
    UtcConvert = 7 # 7 hours for PDT, 8 hours for PST
    if CLRfile.endswith('.csv'):
        CLR = pd.read_csv(rawF + CLRfile, header = 0,index_col = False)
    else:
        print('CLR file format error')        
    CLR['timeUtc'] = pd.to_datetime(CLR['timeUtc'],format='%Y-%m-%d'+'T'+'%H:%M:%S'+'.000Z')
    CLR['datetime'] = CLR['timeUtc'] - pd.Timedelta(hours = UtcConvert)
    CLR.index = CLR['datetime']
    CLR = CLR[['pm1MassConcRaw[ug/m3]','pm2_5MassConcRaw[ug/m3]','pm10MassConcRaw[ug/m3]','no2ConcRaw[ppb]',
               'temperature[degC]','relHumidity[%]','tVocConc[ppb]','eCo2Conc[ppb]']]
    CLR.columns = ['pm1_ugm3','pm25_ugm3','pm10_ugm3','no2_ppb','temp_degC','rh_pct',
                  'tVoc_ppb','eCO2_ppb']
    return CLR


def process_CLR(CLRfile,ind):
    fname = extractName(CLRfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    
    pad = 360
    rawCLR = read_CLR(CLRfile)
    rawCLR = rawCLR[(rawCLR.index>start-pd.Timedelta(seconds=pad)) & (rawCLR.index<stop+pd.Timedelta(seconds=pad))]
    smCLR = rawCLR.resample('s').interpolate(method='time',axis=0, limit_area = 'inside')
    smCLR.index = smCLR.index.shift(-1, freq='30s')
    smCLR = smCLR.rolling('60s').mean()
    CLR = pd.merge(ind,smCLR, left_index=True, right_index=True, how='left')
    
    colmapper = {'pm1_ugm3':'{}_{}_PM1'.format(instr, loc),
             'pm25_ugm3':'{}_{}_PM25'.format(instr, loc),
             'pm10_ugm3':'{}_{}_PM10'.format(instr, loc),
             'no2_ppb':'{}_{}_NO2'.format(instr, loc),
             'temp_degC':'{}_{}_T'.format(instr, loc),
             'rh_pct':'{}_{}_RH'.format(instr, loc),
             'tVoc_ppb':'{}_{}_tVOC'.format(instr, loc),
             'eCO2_ppb':'{}_{}_eCO2'.format(instr, loc)}
    CLR.rename(columns = colmapper, inplace=True)
    CLR.fillna('NA',inplace=True)
    return CLR


##############################################################################################
# 3.4 FRM - Formaldehyde: 30min data (backfill)
##############################################################################################
def read_FRM(FRMfile): # .csv, .xlsx
    readParamsDict = {'header': 0, 'index_col': False}    
    header = ['mode', 'sampnum', 'datanum', 
              'date', 'time', 'frm_ppb', 'frm_ugm3', 
              'temp_degC', 'temp_degF', 'rh_pct', 'period_min']
    if FRMfile[-4:] == '.csv':  
        df = pd.read_csv(rawF + FRMfile, names=header, **readParamsDict)
    elif FRMfile[-5:] == '.xlsx':
        df = pd.read_excel(rawF + FRMfile, names=header, **readParamsDict)
        
    #df['date'] = pd.to_datetime(df['date'],format = dtFormatString(df['date']))
    df.index = pd.to_datetime(df.date.astype(str) + ' ' + df.time.astype(str))
    df.loc[df['frm_ppb']=='<0','frm_ppb'] = 0
    df.loc[df['frm_ppb']=='Baseline','frm_ppb'] = None
    cols_to_use = ['frm_ppb']
    df = df[cols_to_use]
    df = df.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()
    return df


def process_FRM(FRMfile, ind):    
    fname = extractName(FRMfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    pad = 40 # minutes
    rawFRM = read_FRM(FRMfile)
    rawFRM = rawFRM[(rawFRM.index>start-pd.Timedelta(minutes=pad)) & (rawFRM.index<stop+pd.Timedelta(minutes=pad))]
    FRM = pd.merge(ind, rawFRM.resample('s').bfill(), left_index=True, right_index=True, how='left')
    colmapper = {'frm_ppb':'{}_{}'.format(instr, loc)}
    FRM.rename(columns = colmapper, inplace=True)
    FRM.fillna('NA', inplace=True)
    return FRM


##############################################################################################
# 3.5 IBU - iButton T: 1-2min data (interpolate)
##############################################################################################
def read_IBU(IBUfile): # .csv, .xlsx
    if IBUfile.endswith('csv'):
        if int(home_id) in range(200,500):
            IBU = pd.read_csv(rawF+IBUfile,names = ['datetime','temp_degC'],skiprows=8,index_col=None)
            IBU = IBU.iloc[:-1]
            IBU['datetime'] = fromExcelDate(IBU['datetime'])    
            # IBU['datetime'] = pd.to_datetime(IBU['datetime'], format=dtFormatString(IBU['datetime']))
        elif int(home_id) in range(900,1000):
            header = ['datetime', 'unit', 'temp_degC']    
            rows = findHeader(rawF + IBUfile, 'Value')
            IBU = pd.read_csv(rawF + IBUfile, skiprows=rows, header=None, names=header, engine='python', skipfooter=1)  
            IBU['datetime'] = pd.to_datetime(IBU['datetime'],format = dtFormatString(IBU['datetime']))
            if IBU.loc[:,'unit'][0].lower() != 'c':
                print("IBU unit error")
    elif IBUfile.endswith('xlsx'):
        if int(home_id) in range (200,500):
            IBUi = pd.read_excel(rawF + IBUfile, header=None, skiprows=0, index_col=False)
            rows = IBUi.loc[IBUi.iloc[:,0]=='Date - Time'].index[0]  
            IBU = pd.read_excel(rawF + IBUfile, skiprows=rows, header=0, names=['datetime','temp_degC'])
            IBU = IBU[IBU['datetime'].astype(str) != '-end-']
            if dtFormatInspector(IBU.loc[:,'datetime'][5], '%m/%d/%y %H:%M:%S.%f'):
                IBU['datetime'] = pd.to_datetime(IBU['datetime'], format='%m/%d/%y %H:%M:%S.%f').dt.round('1s')    
            elif IBU.loc[:,'datetime'].dtypes == float:
                IBU['datetime'] = fromExcelDate(IBU['datetime'])    
            else:
                IBU['datetime'] = pd.to_datetime(IBU['datetime'], format=dtFormatString(IBU['datetime']))
    IBU.index = IBU.datetime 
    IBU = IBU['temp_degC']
    return IBU


def process_IBU(IBUfile, ind):
    fname = extractName(IBUfile)
    home_id, instr, loc = fname.split('-',2)
    
    pad = 240
    rawIBU = read_IBU(IBUfile)
    rawIBU = rawIBU[(rawIBU.index>start-pd.Timedelta(seconds=pad)) & (rawIBU.index<stop+pd.Timedelta(seconds=pad))]

    IBU = pd.merge(ind, rawIBU, left_index=True, right_index=True, how='outer')
    IBU = IBU.interpolate(method='time', axis=0, limit_area='inside')
    IBU = pd.merge(ind, IBU, left_index=True, right_index=True, how='left')
    
    colmapper = {'temp_degC':'{}_{}'.format(instr, loc)}
    IBU.rename(columns = colmapper, inplace=True)
    IBU.fillna('NA', inplace=True)
    return IBU


##############################################################################################
# 3.6 MTR - Motor on/off state: event record (convert) or 1min state data 
##############################################################################################
def read_MTR(MTRfile): # KVZNE 'event' type record - be careful with duplicates
    if MTRfile.endswith('csv'):
        MTR = pd.read_csv(rawF+MTRfile, header = None, sep=',',skiprows=findHeader(rawF+MTRfile,'Date')).iloc[:,0:3]
        MTR.columns = ['datanum','datetime','event/state']
        MTR['datetime'] = pd.to_datetime(MTR['datetime'], format = dtFormatString(MTR['datetime']))
        if checkLogType(MTR['datetime']) == 'event':
            start = MTR['datetime'].iloc[0]
            stop = MTR['datetime'].iloc[-1]
            new_idx = pd.DataFrame(index = pd.date_range(start, stop, freq='s'))
            MTR = pd.merge(new_idx, MTR, how='left', left_index=True, right_on='datetime').fillna(method='ffill')
            MTR.loc[:,'motor_pct_on'] = MTR.loc[:,'event/state'] * 100 
            MTR.index = MTR['datetime']
            MTR = MTR['motor_pct_on']
            print('MTR logtype: event')
        elif checkLogType(MTR['datetime']) == 'state':
            print('MTR logtype: state')
            MTR['motor_pct_on'] = MTR['event/state']
            MTR.index = MTR['datetime']
            MTR = MTR['motor_pct_on']
    else:
        print('MTR file format error')
    return MTR


def process_MTR(MTRfile, ind):
    fname = extractName(MTRfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    
    pad = 300 # seconds
    rawMTR = read_MTR(MTRfile)
    rawMTR = rawMTR[(rawMTR.index>start-pd.Timedelta(seconds=pad)) & (rawMTR.index<stop+pd.Timedelta(seconds=pad))]    
    MTR = pd.merge(ind, rawMTR, left_index=True, right_index=True, how='left')
    
    colmapper = {'motor_pct_on':'{}_{}'.format(instr, loc)}    
    MTR.rename(columns = colmapper, inplace=True)
    MTR.fillna('NA', inplace=True)
    return MTR

##############################################################################################
# 3.7 PLD - Plug Load: 1min data
##############################################################################################
def read_PLD(PLDfile): # KVZNE 'event' type record - be careful with duplicates
    if PLDfile.endswith('csv'):
        PLD = pd.read_csv(rawF+PLDfile, header = None, sep=',',skiprows=findHeader(rawF+PLDfile,'Logged')).iloc[:,0:3]
        PLD.columns = ['datanum','datetime','active_power_W']
        PLD['datetime'] = pd.to_datetime(PLD['datetime'], format = dtFormatString(PLD['datetime']))
        PLD.index = PLD['datetime']
        PLD = PLD['active_power_W']
    else:
        print('PLD file format error')
    return PLD


def process_PLD(PLDfile, ind):
    fname = extractName(PLDfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    
    pad = 120
    rawPLD = read_PLD(PLDfile)
    rawPLD = rawPLD[(rawPLD.index>start-pd.Timedelta(seconds=pad)) & (rawPLD.index<stop+pd.Timedelta(seconds=pad))]
    rawPLD = rawPLD.resample('s').bfill()
    PLD = pd.merge(ind, rawPLD, left_index=True, right_index=True, how='left')
    
    colmapper = {'active_power_W':'{}_{}'.format(instr, loc)}
    PLD.rename(columns = colmapper, inplace=True)
    PLD.fillna('NA', inplace=True)
    return PLD


##############################################################################################
# 3.8 PMI - Indoor PM2.5: 1-2min data 
##############################################################################################
def read_PMI(PMIfile):
    if home_id == '906':
        PMIStart = pd.to_datetime('02/12/2019 3:45:57 PM',format = '%m/%d/%Y %I:%M:%S %p')
        PMI = pd.read_excel(rawF+PMIfile, skiprows = 24)
        PMI.index = pd.to_datetime(PMI['datetime'])
        PMI['pm_in_ugm3'] = pd.to_numeric(PMI[' "ug/m3"'])
        PMI = PMI['pm_in_ugm3']
    elif int(home_id) in range(200,400):
        PMI = pd.read_csv(rawF+PMIfile, header=0)
        PMI.drop_duplicates(subset='Time', keep='first',inplace = True)
        PMI['datetime'] = pd.to_datetime(PMI['Time'],format = dtFormatString(PMI['Time']))
        PMI['pm_in_ugm3'] = pd.to_numeric(PMI['Conc'])
        PMI.index = PMI['datetime']
        PMI = PMI['pm_in_ugm3']
    elif int(home_id) in range(400,500):
        PMI = pd.read_excel(rawF+PMIfile, header=0)
        PMI.drop_duplicates(subset='Time', keep='first',inplace = True)
        PMI['datetime'] = pd.to_datetime(PMI['Time'],format = dtFormatString(PMI['Time']))
        PMI['pm_in_ugm3'] = pd.to_numeric(PMI['Conc'])
        PMI.index = PMI['datetime']
        PMI = PMI['pm_in_ugm3']
    elif int(home_id) in range(900,1000):
        IndStartDate = findHeader(rawF+PMIfile,'Test Start Date')
        IndStartTime = findHeader(rawF+PMIfile,'Test Start Time')
        PMIStartDate = pd.read_csv(rawF + PMIfile, nrows = IndStartDate-1).iloc[-1,1]
        PMIStartTime = pd.read_csv(rawF + PMIfile, nrows = IndStartTime-1).iloc[-1,1]
        PMIStart = pd.to_datetime(PMIStartDate + ' ' + PMIStartTime, format = '%m/%d/%Y %I:%M:%S %p')

        PMI = pd.read_csv(rawF + PMIfile, header = findHeader(rawF + PMIfile,'Elapsed Time [s]')-2).iloc[:,0:2]
        PMI['Elapsed Time [s]'] = pd.to_numeric(PMI['Elapsed Time [s]'])
        PMI['Elapsed Time [s]'] = pd.to_timedelta(PMI['Elapsed Time [s]'],unit='s')
        PMI['datetime'] = PMIStart + PMI['Elapsed Time [s]']
        PMI.index = PMI['datetime']
        PMI['pm_in_ugm3'] = pd.to_numeric(PMI['Mass [mg/m3]'])*1000
        PMI = PMI['pm_in_ugm3']
    return PMI


def process_PMI(PMIfile,ind):
    fname = extractName(PMIfile)    
    home_id, instr, loc = fname.split('-',2)
    
    pad = 600
    rawPMI = read_PMI(PMIfile)
    rawPMI = rawPMI[(rawPMI.index>start-pd.Timedelta(seconds=pad)) & (rawPMI.index<stop+pd.Timedelta(seconds=pad))]
    
#     smPMI = rawPMI.resample('s').interpolate(method='time', axis=0, limit_area='inside')
#     smPMI.index = smPMI.index.shift(-1, freq='30s')
#     smPMI = smPMI.rolling('60s').mean()
    smPMI = rawPMI.resample('s').bfill()
    PMI = pd.merge(ind, smPMI, left_index=True, right_index=True, how='left')
    
    colmapper = {'pm_in_ugm3':'PMin'}
    PMI.rename(columns = colmapper, inplace=True)
    PMI.fillna('NA', inplace=True)
    return PMI


##############################################################################################
# 3.9 PMO - Outdoor PM2.5: 1-2min data (interpolate)
##############################################################################################
def read_PMO(PMOfile): 
    if int(home_id) in range(900,1000):
        IndStartDate = findHeader(rawF+PMOfile,'Test Start Date')
        IndStartTime = findHeader(rawF+PMOfile,'Test Start Time')
        PMOStartDate = pd.read_csv(rawF + PMOfile, nrows = IndStartDate-1).iloc[-1,1]
        PMOStartTime = pd.read_csv(rawF + PMOfile, nrows = IndStartTime-1).iloc[-1,1]
        PMOStart = pd.to_datetime(PMOStartDate + ' ' + PMOStartTime, format = '%m/%d/%Y %I:%M:%S %p')
        PMO = pd.read_csv(rawF + PMOfile, header = findHeader(rawF + PMOfile,'Elapsed Time [s]')-2).iloc[:,0:2]
        PMO['Elapsed Time [s]'] = pd.to_numeric(PMO['Elapsed Time [s]'])
        PMO['Elapsed Time [s]'] = pd.to_timedelta(PMO['Elapsed Time [s]'],unit='s')
        PMO['datetime'] = PMOStart + PMO['Elapsed Time [s]']
        PMO.index = PMO['datetime']
        PMO['pm_out_ugm3'] = pd.to_numeric(PMO['Mass [mg/m3]'])*1000
        PMO = PMO['pm_out_ugm3']
    elif int(home_id) in range(200,400): # USE Smp1Min
        PMO = pd.read_csv(rawF + PMOfile, skiprows = 4)
        PMO.columns = ['datetime','RN','pm_out_ugm3','flowrate','T','RH','BP','Status']
        PMO = PMO.loc[PMO['flowrate']!=0,:]  # Do not import data when flowrate = 0 (zeroing values)
        PMO = PMO.loc[PMO['flowrate']!='0',:]
        PMO['datetime'] = pd.to_datetime(PMO['datetime'],format = dtFormatString(PMO['datetime']))
        PMO = PMO.loc[PMO['pm_out_ugm3']!='NAN',:]
        PMO['pm_out_ugm3'] = pd.to_numeric(PMO['pm_out_ugm3'])
        PMO.index = PMO['datetime']
        PMO = PMO['pm_out_ugm3']
    elif int(home_id) in range(400,500): # USE "scaled series - avg"
        PMO = pd.read_excel(rawF + PMOfile, skiprows = 2, names = ['datanum','datetime','pm_out_ugm3'])
        PMO = PMO.dropna()
        PMO['datetime'] = pd.to_datetime(PMO['datetime'],format = dtFormatString(PMO['datetime']))
        PMO.index = PMO['datetime']
        PMO = PMO['pm_out_ugm3']
    return PMO


def process_PMO(PMOfile,ind):
    fname = extractName(PMOfile)    
    home_id, instr, loc = fname.split('-',2)
    
    pad = 600
    rawPMO = read_PMO(PMOfile)
    rawPMO = rawPMO[(rawPMO.index>start-pd.Timedelta(seconds=pad)) & (rawPMO.index<stop+pd.Timedelta(seconds=pad))]
    
#     smPMO = rawPMO.resample('s').interpolate(method='time', axis=0, limit_area='inside')
#     smPMO.index = smPMO.index.shift(-1, freq='30s')
#     smPMO = smPMO.rolling('60s').mean()
    smPMO = rawPMO.resample('s').bfill()
    PMO = pd.merge(ind, smPMO, left_index=True, right_index=True, how='left')
    
    colmapper = {'pm_out_ugm3':'PMout'}
    PMO.rename(columns = colmapper, inplace=True)
    PMO.fillna('NA', inplace=True)
    return PMO


##############################################################################################
# 3.10 RAD/RDSTR - Radon: hourly data 
##############################################################################################
def read_RAD(RADfile): # .csv, .xlsx
    if RADfile.endswith('csv'):
        home_id, instr, loc = re.split('-', RADfile[:-4])[0:3]
        RAD = pd.read_csv(rawF+RADfile, names = ['datanum', 'Rn_pCiL', 'datetime', 'SN'], skiprows=1)
    if RADfile.endswith('xlsx'):
        home_id, instr, loc = re.split('-', RADfile[:-5])[0:3]
        RAD = pd.read_excel(rawF+RADfile, names = ['datanum', 'Rn_pCiL', 'datetime', 'SN'], skiprows=1)
    RAD['datetime'] = pd.to_datetime(RAD['datetime'],format = dtFormatString(RAD['datetime']))
    serial = pd.to_numeric(RAD.SN[5])
    RAD['Rn_pCiL'] = pd.to_numeric(RAD['Rn_pCiL'])
    RAD = RAD.loc[RAD['Rn_pCiL']!=serial,:]
    RAD.index = RAD['datetime']
    RAD = RAD['Rn_pCiL']
    return RAD

def process_RAD(RADfile, ind):
    fname = extractName(RADfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    pad = 120 # minutes
    rawRAD = read_RAD(RADfile)[start-pd.Timedelta(minutes=pad):stop+pd.Timedelta(minutes=pad)]
    RAD = pd.merge(ind, rawRAD.resample('s').ffill(), left_index=True, right_index=True, how='left')

    colmapper = {'Rn_pCiL':'Rn_{}'.format(loc)}
    RAD.rename(columns = colmapper, inplace=True)
    RAD.fillna('NA', inplace=True)
    return RAD


##############################################################################################
# 3.11 STA - Door close/open state: event record (convert) or 1min state data 
##############################################################################################
def read_STA(STAfile): # KVZNE 'event' type record - be careful with duplicates
    if STAfile.endswith('csv'):
        STA = pd.read_csv(rawF+STAfile, header = None, sep=',',skiprows=findHeader(rawF+STAfile,'Date')).iloc[:,0:3]
        STA = STA[~STA.isin(['Logged'])]
        STA.columns = ['datanum','datetime','event/state']
        STA['datetime'] = pd.to_datetime(STA['datetime'], format = dtFormatString(STA['datetime']))
    elif STAfile.endswith('xlsx'):
        STA = pd.read_excel(rawF+STAfile, usecols = [0,1,2], names = ['datanum','datetime','event/state'])
        STA = STA[~STA.isin(['Logged'])]
        STA['datetime'] = pd.to_datetime(STA['datetime'], format = dtFormatString(STA['datetime']))
    else:
        print('STA file format error')
    if checkLogType(STA['datetime']) == 'event':
        start = STA['datetime'].iloc[0]
        stop = STA['datetime'].iloc[-1]
        new_idx = pd.DataFrame(index = pd.date_range(start, stop, freq='s'))
        STA = pd.merge(new_idx, STA, how='left', left_index=True, right_on='datetime').fillna(method='ffill')
        STA.loc[:,'state_pct_closed'] = STA.loc[:,'event/state'] * 100 
        print('STA logtype: event')
    elif checkLogType(STA['datetime']) == 'state':
        print('STA logtype: state')
        STA['state_pct_closed'] = STA['event/state']
    STA.index = STA['datetime']
    STA = STA['state_pct_closed']
    return STA


def process_STA(STAfile, ind):
    fname = extractName(STAfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    
    pad = 120
    rawSTA = read_STA(STAfile)
    rawSTA = rawSTA[(rawSTA.index>start-pd.Timedelta(seconds=pad)) & (rawSTA.index<stop+pd.Timedelta(seconds=pad))]
    smSTA = rawSTA.resample('s').bfill()
    STA = pd.merge(ind, smSTA, left_index=True, right_index=True, how='left')
    
    colmapper = {'state_pct_closed':'{}_{}'.format(instr, loc)}    
    STA.rename(columns = colmapper, inplace=True)
    STA.fillna('NA', inplace=True)
    return STA

##############################################################################################
# 3.12 TRH/TRO - HOBO T/RH: 1min data (fill NA)
##############################################################################################
def read_TRH(TRHfile):
    if TRHfile.endswith('csv'):
        home_id, instr, loc = re.split('-', TRHfile[:-4])[0:3]        
        TRH = pd.read_csv(rawF+TRHfile, usecols = [0,1,2,3],names = ['datanum','datetime','temp_degF','rh_pct'],
                          skiprows = 2, index_col = None)
    elif TRHfile.endswith('xlsx'):
        home_id, instr, loc = re.split('-', TRHfile[:-5])[0:3] 
        TRH = pd.read_excel(rawF+TRHfile, usecols = [0,1,2,3],names = ['datanum','datetime','temp_degF','rh_pct'],
                          skiprows = 2, index_col = None)
    else:
        print('TRH file format error')
    TRH = TRH[~TRH.isin(['Logged'])]
    TRH = TRH[['datetime','temp_degF','rh_pct']].dropna(axis=0,thresh=2)

    TRH['datetime'] = pd.to_datetime(TRH['datetime'],format=dtFormatString(TRH['datetime']))
    TRH['temp_degC'] = temperatureConverter(TRH['temp_degF'],'f')
    TRH = TRH[['datetime','temp_degC','rh_pct']]          
    if TRH['datetime'].iloc[-1] < stop:
        newdatetime = TRH['datetime'].iloc[-1]+pd.Timedelta(minutes=1)
        newrow = pd.Series({'datetime' : newdatetime , 'temp_degC' : None, 'rh_pct': None})
        TRH = TRH.append(newrow,ignore_index = True)       
        
    TRH.index = TRH['datetime']
    TRH = TRH[['temp_degC','rh_pct']]
    return TRH


def process_TRH(TRHfile, ind):
    fname = extractName(TRHfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]    
    pad = 60
    rawTRH = read_TRH(TRHfile)[start-pd.Timedelta(seconds=pad):stop+pd.Timedelta(seconds=pad)]
    TRH = pd.merge_asof(ind, rawTRH, left_index=True, right_index=True, direction='nearest')    
    colmapper = {'temp_degC':'{}_T'.format(loc),'rh_pct':'{}_RH'.format(loc)}    
    TRH.rename(columns = colmapper, inplace=True)
    TRH.fillna('NA', inplace=True)
    return TRH


##############################################################################################
# 3.13 UFP - DISCmini: number of UltraFineParticle
# insert last row with "NA" for 901
##############################################################################################
def read_UFP(UFPfile): # DISCmini - Betel(2units,IN1),number of particles
    if int(home_id) in range(400,500):
        UFP = pd.read_excel(rawF+UFPfile, header = 5,index_col = None)
    elif int(home_id) in range(900,1000):
        UFP = pd.read_csv(rawF+UFPfile, sep=',')
    
    UFP['datetime'] = pd.to_datetime(UFP['Time'], format = '%Y-%m-%d %H:%M:%S')
    
    if UFP['datetime'].iloc[-1] < stop:
        newdatetime = UFP['datetime'].iloc[-1]+pd.Timedelta(minutes=1)
        newrow = pd.Series({'datetime':newdatetime, 'Number':None})
        UFP = UFP.append(newrow,ignore_index = True)
    UFP.index = UFP['datetime']
    UFP['ufp_count'] = pd.to_numeric(UFP['Number'])
    UFP = UFP['ufp_count'] 
    return UFP


def process_UFP(UFPfile,ind):
    fname = extractName(UFPfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]   
    
    rawUFP = read_UFP(UFPfile)
    
    pad = 60 # seconds
    rawUFP = rawUFP[(rawUFP.index>start-pd.Timedelta(seconds=pad)) & (rawUFP.index<stop+pd.Timedelta(seconds=pad))]    
    rawUFP = rawUFP.dropna()
    smUFP = rawUFP.resample('s').interpolate(method='time',axis=0, limit_area='inside')
    smUFP.index = smUFP.index.shift(-1, freq='30s')
    smUFP = smUFP.rolling('60s').mean()
    UFP = pd.merge(df_index, smUFP, left_index=True, right_index=True, how='left')   
   
    colmapper = {'ufp_count':'UFP_{}'.format(loc)}
    UFP.rename(columns=colmapper,inplace=True)
    UFP.fillna('NA', inplace=True)
    return UFP


##############################################################################################
# 3.13 WTR/WX - Weather Underground Outdoor T/RH: hourly data (interpolation)
##############################################################################################
def read_WTR(WTRfile): # .xlsx, same as WX
    WTR = pd.read_excel(rawF+WTRfile, header=0, index_col = False)
    WTR['datetime'] = WTR['Date'] + pd.to_timedelta(WTR['Time'].astype(str))    
    WTR['temp_out_degC'] = temperatureConverter(WTR['Temperature (F)'],'f')
    WTR.rename({'Humidity (%)':'rh_out_pct'},axis = 1, inplace = True)
    WTR.index = WTR['datetime']
    WTR = WTR[['temp_out_degC','rh_out_pct']]
    return WTR


def process_WTR(WTRfile,ind):
    fname = extractName(WTRfile)    
    home_id, instr, loc = re.split('-', fname)[0:3]
    
    rawWTR = read_WTR(WTRfile)
    WTR = pd.merge(ind, rawWTR, left_index=True, right_index=True, how='outer')
    WTR = WTR.interpolate(method='time', axis=0, limit_area='inside')
    WTR = pd.merge(ind, WTR, left_index=True, right_index=True, how='left')
    
    colmapper = {'temp_out_degC':'{}_T'.format(loc),'rh_out_pct':'{}_RH'.format(loc)}    
    WTR.rename(columns = colmapper, inplace=True)
    WTR.fillna('NA', inplace=True)
    return WTR



In [12]:
'''
##########################################################################################################
4. Output: Duplicate Inspection and DataTable Writing
##########################################################################################################
'''
dataTable = df_index
countDone = 0
for instr, fList in instrDict.items():    
    if len(fList) > 0:
        for f in fList:
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            home_id, instr, loc = re.split('-', f)[0:3]
            print(f)
            df = processFile(f, df_index, instr, home_id, week)
            if df[df.index.duplicated(keep='first')].shape[0] == 0:
                print('No duplicates')
            else:
                print ('Has duplicates XXXXXXXXXXXXXXXXXXXXXXXXXXXX')
            dataTable = pd.merge(dataTable, df, how='left', left_index=True, right_index=True)
            countDone += 1
            print(str(countDone) +'/' + str(count) + ' files done')
            del df
print('--------------------END--------------------')
dataTable 

if week == 1 or week == None:
    dataTable.to_csv(outF + 'DataTable{}_Week1_ON.csv'.format(home_id))
elif week == 2:
    dataTable.to_csv(outF + 'DataTable{}_Week2_OFF.csv'.format(home_id))

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
224-ANM-BA2.xls
No duplicates
1/28 files done
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
224-ANM-BA1.xls
No duplicates
2/28 files done
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
224-ANM-KIT.xls
No duplicates
3/28 files done
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
224-ANM-AS1.xls
No duplicates
4/28 files done
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
224-AVP-IN1.csv


IndexError: single positional indexer is out-of-bounds

In [ ]:
# for summary writing use

In [19]:
dataTable = dataTable.apply(pd.to_numeric, errors='coerce')

'''
##########################################################################################################
5. Output: Summary
##########################################################################################################
'''
##############################################################################################
# Load activity log
##############################################################################################
actF = '/Users/yuhanwang/Desktop/LBNL/BA/BA_Datatable/' # activity log folder
if int(home_id) in range(200,400):
    act_fname = 'Occupancy_PNNL.xlsx'
elif int(home_id) in range(400,500):
    act_fname = 'Occupancy_FSEC.xlsx'
act_df = pd.read_excel(actF + act_fname)
act_df.loc[:,'Date'] = pd.to_datetime(act_df.Date)


##############################################################################################
# Define Support Function
##############################################################################################
def initiate_occupancy_df(filtered_df):
    act_df = pd.read_excel(actF + act_fname)
    act_df.loc[:,'Date'] = pd.to_datetime(act_df.Date)
    list1 = pd.to_timedelta(np.arange(0,24), unit='h').tolist()
    list2 = [list1[i] + pd.Timedelta(minutes=59, seconds=59) for i in range(len(list1))]
    list3 = [(list1[x], list2[x]) for x in range(len(list1))]
    hr_dict = dict(zip(act_df.columns[3:27].tolist(), list3))
    occupy_idx = []
    for ind, row in filtered_df.iterrows():
        date = pd.to_datetime(row.name)
        occupyList = row[row==True].index.tolist()
        for i in occupyList:
            t1 = date + hr_dict[i][0]
            t2 = date + hr_dict[i][1]
            occupy_idx = occupy_idx + [str(ts) for ts in pd.date_range(t1, t2, freq='min')]
            del(t1, t2)
        del(occupyList, date)
    df = pd.DataFrame(index = pd.to_datetime(occupy_idx))
    return df


##############################################################################################
# Extract occupied slice
##############################################################################################
if int(home_id) in set(range(900,1000)): # KVZNE study
        startrow = 'Field_StartDate'
        stoprow = 'Field_EndDate'
elif int(home_id) in set(range(200,500)): # FSEC
        if week == 1 or week == None:
            startrow = 'Field_StartDate_Week1'
            stoprow = 'Field_EndDate_Week1'
        elif week == 2:
            startrow = 'Field_StartDate_Week2'
            stoprow = 'Field_EndDate_Week2'
else:
    print("change home_id")

start = pd.to_datetime(ts_df.loc[startrow, int(home_id)], format = '%Y-%m-%d') + pd.Timedelta(hours=18)
stop = pd.to_datetime(ts_df.loc[stoprow, int(home_id)], format = '%Y-%m-%d') + pd.Timedelta(hours=9)
startday = start.strftime('%Y-%m-%d')
stopday = stop.strftime('%Y-%m-%d')

homerows = act_df.Home_ID == int(home_id)
timecols = lambda df: df.columns[2:27]
nightcols = act_df.columns[3:27].tolist()[0:5]

allSlice = act_df.loc[homerows, timecols]
allSlice = allSlice.set_index('Date', drop=True)
allSlice = allSlice[startday:stopday] > 0
nightSlice = allSlice.loc[:, nightcols]

occupyAll_idx = initiate_occupancy_df(allSlice)[start:stop]
occupyNight_idx = initiate_occupancy_df(nightSlice)[start:stop]

label = home_id # caution
results = pd.DataFrame(index=[label])
print(home_id + ' week:' + str(week))

##############################################################################################
# a. Indoor CO2 (Living Room)
##############################################################################################
col_a = 'AVP_IN2_CO2'

# -> (1) 24/7 + occupied
if len(occupyAll_idx)==0:
    occupyAll_idx = initiate_occupancy_df(allSlice.replace(allSlice,True))[start:stop]
co2_indoor = occupyAll_idx    
co2_indoor = pd.merge(co2_indoor, dataTable[col_a].to_frame(), left_index=True, right_index=True, how='left')
co2_indoor_mean_occupied = co2_indoor.iloc[:,0].mean()
print('CO2_IN: '+ str(co2_indoor_mean_occupied))
# -> (2) 24/7
co2_indoor_mean = dataTable[col_a].mean()

##############################################################################################
# b. Bedroom CO2 (Master Bedroom)
##############################################################################################
col_b = 'AVP_BR1_CO2'

# Hr0->Hr4 + occupied
if len(occupyNight_idx)==0:
    occupyNight_idx = initiate_occupancy_df(nightSlice.where(nightSlice==True, True))[start:stop]
    
co2_bedroom = occupyNight_idx
co2_bedroom = pd.merge(co2_bedroom, dataTable[col_b].to_frame(), left_index=True, right_index=True, how='left')
co2_bedroom_mean_occupied = co2_bedroom.iloc[:,0].mean()

print('CO2_BR1: '+ str(co2_bedroom_mean_occupied))

##############################################################################################
# c. Indoor Temperature (Living Room)
##############################################################################################
col_c = 'AVP_IN1_T'

temp_indoor = dataTable[col_c]
c_q1 = temp_indoor.quantile(0.01)
c_q99 = temp_indoor.quantile(0.99)
temp_indoor_mean = temp_indoor[(temp_indoor>=c_q1) & (temp_indoor<=c_q99)].mean()

print('T_RangeLower (F): '+str(temperatureConverter(c_q1,'c')))
print('T_RangeUpper (F): '+str(temperatureConverter(c_q99,'c')))
print('T_Mean (C): '+str(temp_indoor_mean))

##############################################################################################
# d. Bedroom Temperature (Master Bedroom)
##############################################################################################

##############################################################################################
# e. Outdoor Temperature
##############################################################################################
col_e = 'OUT_T'

temp_outdoor = dataTable[col_e]
e_q1 = temp_outdoor.quantile(0.01)
e_q99 = temp_outdoor.quantile(0.99)
temp_outdoor_mean = temp_outdoor[(temp_outdoor>=e_q1) & (temp_outdoor<=e_q99)].mean()
print('OUT_T (C): '+str(temp_outdoor_mean))

##############################################################################################
# f. Indoor RH (Living Room)
##############################################################################################
col_f = 'AVP_IN1_RH'

rh_indoor = dataTable[col_f]
f_q1 = rh_indoor.quantile(0.01)
f_q99 = rh_indoor.quantile(0.99)
rh_indoor_mean = rh_indoor[(rh_indoor>=f_q1) & (rh_indoor<=f_q99)].mean()

print('RH_RangeLower: '+str(f_q1))
print('RH_RangeUpper: '+str(f_q99))
print('RH_Mean: '+str(rh_indoor_mean))

##############################################################################################
# g. Bedroom RH (Master Bedroom)
##############################################################################################


##############################################################################################
# h. Outdoor RH
##############################################################################################
col_h = 'OUT_RH'

rh_outdoor = dataTable[col_h]
h_q1 = rh_outdoor.quantile(0.01)
h_q99 = rh_outdoor.quantile(0.99)
rh_outdoor_mean = rh_outdoor[(rh_outdoor>=h_q1) & (rh_outdoor<=h_q99)].mean()
print('OUT_RH (C): '+str(rh_outdoor_mean))

##############################################################################################
# g. Radon
##############################################################################################
col_g = 'Rn_BS1'

radon = dataTable[col_g]
g_q1 = radon.quantile(0.01)
g_q99 = radon.quantile(0.99)
radon_mean = radon[(radon>=g_q1) & (radon<=g_q99)].mean()
print('Radon: ' +str(radon_mean))

211 week:2
CO2_IN: 1216.7377843872928
CO2_BR1: 1262.4652024677769


KeyError: 'AVP_IN1_T'